### Neural network for detecting text on kern (yolov5 and Roboflow)
  Neural network was learned on database, which includes photos with different filters. The main idea is to crop the bboxes with text and then to read the text using pytesseract. The goal is to make the task easier for Pytesseract by decreasing the area for text recognition. You can use this notebook to train neural network on your own database

In [ ]:
%cd yolov5 ##-----Start working in yolov5 directory

In [ ]:
## ----Installing yolov5
!git clone https://github.com/ultralytics/yolov5 
%cd yolov5
!git reset --hard 886f1c03d839575afecb059accf74296fad395b6 
!pip install -qr requirements.txt 

In [ ]:

## -----Imports
import torch
from utils.google_utils import gdrive_download

from IPython.core.magic import register_line_cell_magic

import glob
from IPython.display import Image, clear_output, display
from utils.plots import plot_results

from roboflow import Roboflow
import yaml

## -----Settings torch
clear_output()
print('Setup complete. Using torch %s %s' %(torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))
rf = Roboflow(model_format="yolov5", notebook="Text recognition")

## -----Getting database
!curl -L "https://app.roboflow.com/ds/t14ibAqfCx?key=QQfQW48td3" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

## ----Define number of classes based on YAML
with open("data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

### Customizing writefile and configuring neural network parameters 

In [ ]:
## -----Customize iPython writefile so we can write variables

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))
%%writetemplate custom_yolov5m.yaml

## -----Neural network structure
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

backbone:
## -----[from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2 
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

## -----YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

### Choosing the number of batches and epochs, starting training process

In [ ]:
!python train.py --img 416 --batch 32 --epochs 10000 --data 'data.yaml' --cfg ./custom_yolov5m.yaml --weights '' --name yolov5s_results  --cache

### Plotting statistic results about lerning process and showing the result by examples

In [ ]:
Image(filename='runs/train/yolov5s_results6/results.png', width=1000) ## plotting results
print("GROUND TRUTH TRAINING DATA:")  ## display our ground truth data
Image(filename='runs/train/yolov5s_results6/test_batch0_labels.jpg', width=900)
print("GROUND TRUTH TRAINING DATA:") ## next, display our predicted label data
Image(filename='runs/train/yolov5s_results6/test_batch0_pred.jpg', width=900)

### Cropping the image and getting bboxes with text on example

In [ ]:
% cd yolov5
!python detect.py --weights runs/train/yolov5s_results6/weights/best.pt --img 37 --conf 0.4 --source ./test/images

!python detect3.py --weights runs/train/yolov5s_results6/weights/best.pt --img 37 --conf 0.4 --source test/images/33.jpg
for imageName in glob.glob('runs/detect/exp2426/*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")
    name='runs/detect/exp2426/*.jpg'